In [1]:
import unittest
import numpy as np
import pandas as pd
import ctypes
from customProphet import *

class TestGradient(unittest.TestCase):
    def setUp(self):
        # Load the shared library
        self.lib = ctypes.CDLL('./libgradient.so')
        self.lib.gradient.argtypes = [
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
            ctypes.c_size_t,
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
            ctypes.c_size_t,
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
            ctypes.c_size_t,
            ctypes.c_double,
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
            ctypes.c_size_t,
            ctypes.c_double,
            ctypes.c_double,
            ctypes.c_double,
            ctypes.c_double,
            ctypes.c_double,
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
        ]
        self.lib.gradient.restype = None

        # Load data
        df = pd.read_csv('peyton_manning.csv')

        # Instantiate & initialize a model
        self.model = CustomProphet()
        self.model.y = df['y'].values
        if df['ds'].dtype != 'datetime64[ns]':
            self.model.ds = pd.to_datetime(df['ds'])
        else:
            self.model.ds = df['ds']

        self.model.t_scaled = np.array((self.model.ds - self.model.ds.min()) / (self.model.ds.max() - self.model.ds.min()))
        self.model.T = df.shape[0]

        self.model.scale_period = (self.model.ds.max() - self.model.ds.min()).days
        self.model._normalize_y()
        self.model._generate_change_points()

        self.params = np.ones((47,))
    
    def tearDown(self):
        # This method is called after each test
        print('Test passed: gradient function correctly implemented')

    def test_gradient(self):
        # Convert data to ctypes
        params_ctypes = np.ascontiguousarray(self.params, dtype=np.float64)
        t_scaled_ctypes = np.ascontiguousarray(self.model.t_scaled, dtype=np.float64)
        change_points_ctypes = np.ascontiguousarray(self.model.change_points, dtype=np.float64)
        normalized_y_ctypes = np.ascontiguousarray(self.model.normalized_y, dtype=np.float64)

        # Allocate memory for the gradient output
        grad_output = np.zeros_like(params_ctypes)

        # Call the Python method
        grad_python = self.model._gradient(self.params)

        # Call the C++ function
        self.lib.gradient(
            params_ctypes, len(params_ctypes),
            t_scaled_ctypes, len(t_scaled_ctypes),
            change_points_ctypes, len(change_points_ctypes),
            self.model.scale_period,
            normalized_y_ctypes, len(normalized_y_ctypes),
            self.model.sigma_obs,
            self.model.sigma_k,
            self.model.sigma_m,
            self.model.sigma,
            self.model.tau,
            grad_output
        )

        # Assert that the values are close
        np.testing.assert_allclose(grad_python, grad_output, rtol=1e-5, atol=1e-5, err_msg="The C++ and Python gradient values do not match.")

# Run in a Jupyter notebook
suite = unittest.TestLoader().loadTestsFromTestCase(TestGradient)

# Run the test suite
unittest.TextTestRunner(verbosity=2).run(suite)

# Run in a script
#if __name__ == '__main__':
#    unittest.main()

test_gradient (__main__.TestGradient.test_gradient) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.079s

OK


Test passed: gradient function correctly implemented


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [3]:
import ctypes
import numpy as np
import pandas as pd
from customProphet import *

# Load the shared library for the gradient function
lib = ctypes.CDLL('./libgradient.so')
lib.gradient.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
]
lib.gradient.restype = None

# Load data
df = pd.read_csv('peyton_manning.csv')

# Instantiate & initialize a model
model = CustomProphet()
model.y = df['y'].values
if df['ds'].dtype != 'datetime64[ns]':
    model.ds = pd.to_datetime(df['ds'])
else:
    model.ds = df['ds']

model.t_scaled = np.array((model.ds - model.ds.min()) / (model.ds.max() - model.ds.min()))
model.T = df.shape[0]

model.scale_period = (model.ds.max() - model.ds.min()).days
model._normalize_y()
model._generate_change_points()

params = np.ones((47,))

# Convert data to ctypes
params_ctypes = np.ascontiguousarray(params, dtype=np.float64)
t_scaled_ctypes = np.ascontiguousarray(model.t_scaled, dtype=np.float64)
change_points_ctypes = np.ascontiguousarray(model.change_points, dtype=np.float64)
normalized_y_ctypes = np.ascontiguousarray(model.normalized_y, dtype=np.float64)
grad_out = np.zeros((47,), dtype=np.float64)

***
## Speed Test

In [4]:
%%timeit -r 50 -n 50
lib.gradient(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau,
    grad_out
)

411 µs ± 35.5 µs per loop (mean ± std. dev. of 50 runs, 50 loops each)


In [6]:
%%timeit -r 50 -n 50
model._gradient(params)

1.14 ms ± 244 µs per loop (mean ± std. dev. of 50 runs, 50 loops each)


In [1]:
import unittest
import numpy as np
import pandas as pd
import ctypes
from customProphet import *

class TestGradient(unittest.TestCase):
    def setUp(self):
        # Load the shared library
        self.lib = ctypes.CDLL('./libgradient.so')
        self.lib.gradient.argtypes = [
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
            ctypes.c_size_t,
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
            ctypes.c_size_t,
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
            ctypes.c_size_t,
            ctypes.c_double,
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
            ctypes.c_size_t,
            ctypes.c_double,
            ctypes.c_double,
            ctypes.c_double,
            ctypes.c_double,
            ctypes.c_double,
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
        ]
        self.lib.gradient.restype = None

        # Load data
        df = pd.read_csv('peyton_manning.csv')

        # Instantiate & initialize a model
        self.model = CustomProphet()
        self.model.y = df['y'].values
        if df['ds'].dtype != 'datetime64[ns]':
            self.model.ds = pd.to_datetime(df['ds'])
        else:
            self.model.ds = df['ds']

        self.model.t_scaled = np.array((self.model.ds - self.model.ds.min()) / (self.model.ds.max() - self.model.ds.min()))
        self.model.T = df.shape[0]

        self.model.scale_period = (self.model.ds.max() - self.model.ds.min()).days
        self.model._normalize_y()
        self.model._generate_change_points()

        self.params = np.ones((47,))
    
    def tearDown(self):
        # This method is called after each test
        print('Test passed: gradient function correctly implemented')

    def test_gradient(self):
        # Convert data to ctypes
        params_ctypes = np.ascontiguousarray(self.params, dtype=np.float64)
        t_scaled_ctypes = np.ascontiguousarray(self.model.t_scaled, dtype=np.float64)
        change_points_ctypes = np.ascontiguousarray(self.model.change_points, dtype=np.float64)
        normalized_y_ctypes = np.ascontiguousarray(self.model.normalized_y, dtype=np.float64)

        # Allocate memory for the gradient output
        grad_output = np.zeros_like(params_ctypes)

        # Call the Python method
        grad_python = self.model._gradient(self.params)

        # Call the C++ function
        self.lib.gradient(
            params_ctypes, len(params_ctypes),
            t_scaled_ctypes, len(t_scaled_ctypes),
            change_points_ctypes, len(change_points_ctypes),
            self.model.scale_period,
            normalized_y_ctypes, len(normalized_y_ctypes),
            self.model.sigma_obs,
            self.model.sigma_k,
            self.model.sigma_m,
            self.model.sigma,
            self.model.tau,
            grad_output
        )

        # Assert that the values are close
        np.testing.assert_allclose(grad_python, grad_output, rtol=1e-5, atol=1e-5, err_msg="The C++ and Python gradient values do not match.")

# Run in a Jupyter notebook
suite = unittest.TestLoader().loadTestsFromTestCase(TestGradient)

# Run the test suite
unittest.TextTestRunner(verbosity=2).run(suite)

# Run in a script
#if __name__ == '__main__':
#    unittest.main()

test_gradient (__main__.TestGradient.test_gradient) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.346s

OK


Test passed: gradient function correctly implemented


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [2]:
import ctypes
import numpy as np
import pandas as pd
from customProphet import *

# Load the shared library for the gradient function
lib = ctypes.CDLL('./libgradient.so')
lib.gradient.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
]
lib.gradient.restype = None

# Load data
df = pd.read_csv('peyton_manning.csv')

# Instantiate & initialize a model
model = CustomProphet()
model.y = df['y'].values
if df['ds'].dtype != 'datetime64[ns]':
    model.ds = pd.to_datetime(df['ds'])
else:
    model.ds = df['ds']

model.t_scaled = np.array((model.ds - model.ds.min()) / (model.ds.max() - model.ds.min()))
model.T = df.shape[0]

model.scale_period = (model.ds.max() - model.ds.min()).days
model._normalize_y()
model._generate_change_points()

params = np.ones((47,))

# Convert data to ctypes
params_ctypes = np.ascontiguousarray(params, dtype=np.float64)
t_scaled_ctypes = np.ascontiguousarray(model.t_scaled, dtype=np.float64)
change_points_ctypes = np.ascontiguousarray(model.change_points, dtype=np.float64)
normalized_y_ctypes = np.ascontiguousarray(model.normalized_y, dtype=np.float64)
grad_out = np.zeros((47,), dtype=np.float64)

In [4]:
%%timeit -r 100 -n 100
lib.gradient(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau,
    grad_out
)

413 µs ± 17.9 µs per loop (mean ± std. dev. of 100 runs, 100 loops each)
